In [1]:
import akasha
import os
# import PyPDF2
from time import time
import requests
from bs4 import BeautifulSoup
import json

/home/ychm0304/anaconda3/envs/aks-plus/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = "openai:gpt-4o"

In [3]:
QA = akasha.Doc_QA(model=model, max_doc_len=8000)
summary = akasha.Summary(chunk_size=1000, max_doc_len=7000)

In [ ]:
system_prompt = "你是一個非常擅長寫報告的電池產業專業人員，你會根據指定的格式來生成報告，且內容會完全符合指定的主題。若你無法回答會告訴我原因。"
titles = {
    "全球電池市場概況": [
        "市場規模與增長趨勢",
        "主要市場區域分析",
        "主要企業概況",
    ],
    "技術發展趨勢": [
        "鋰離子電池技術進展",
        "固態電池技術",
        "其他新興電池技術",
    ],
    "應用領域與市場機會": [
        "電動車市場",
        "消費電子產品",
        "能源儲存系統",
        "工業及其他應用",
        "市場需求預測",
        "市場挑戰與機遇",
    ],
    "政策與供應鏈分析": [
        "各國政策支持與法規要求",
        "環保與回收政策",
        "供應鏈與原材料供應",
        "產業佈局與投資趨勢",
    ]
}

links = [
    "https://www.maonline.com.tw/article_inside.php?i=798",
    "https://www.materialsnet.com.tw/DocView.aspx?id=53579",
    "https://ic.tpex.org.tw/introduce.php?ic=E000",
    "https://ieknet.iek.org.tw/ieknews/news_open.aspx?actiontype=ieknews&indu_idno=5&nsl_id=1039fffa4a7d408cb3f5c579762637e1",
    "https://yez.one/post/batterycell",
    "https://www.thenewslens.com/article/181546"
]

theme = "電池產業的發展趨勢"
# titles = ["企業前瞻"]

In [ ]:
def export_dict_to_json(dictionary, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(dictionary, f, ensure_ascii=False, indent=4)

def import_dict_from_json(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

In [ ]:
result = import_dict_from_json("result.json")

In [ ]:
start_time = time()

def generate_report(theme, titles, links, rerun_process=False):
    result = {}
    contexts = []
    for title, subtitle in titles.items():
        format_prompt = f"以{theme}為主題，請你總結撰寫出與\"{title}\"相關的內容，其中需包含{subtitle}，不需要結論，不需要回應要求。"
        for link in links:
            str_len = len("Processing link: " + link) + 1
            print("#" * str_len)
            print("Processing link: ", link)
            print("#" * str_len)

            try:
                response = requests.get(link)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')
                texts = soup.get_text()
            except requests.exceptions.RequestException as e:
                print(f"An error occurred: {e}")
                texts = ''

            contexts.append(
                summary.summarize_articles(
                    articles=texts,
                    format_prompt=format_prompt,
                )
            )

        response = QA.ask_self(
            prompt=f"將此內容以客觀角度進行融合，避免使用\"報告中提到\"相關詞彙，避免修改專有名詞，避免做出總結，\
                        直接撰寫內容，避免回應要求。",
            info=contexts,
            model="openai:gpt-4"
        )
        if rerun_process:
            return response
        result[title] = response
        contexts = []

        print(f"----------integrated summary of {title}----------")
        print(response + "\n")
    return result


result = generate_report(theme, titles, links)
previous_result = ""
for value in result.values():
    previous_result += value

result["內容摘要"] = summary.summarize_articles(
    articles=previous_result,
    format_prompt=f"將內容以{theme}為主題進行摘要，將用字換句話說，意思不變，不需要結論，不需要回應要求。",
    summary_len=500
)

export_dict_to_json(result, "result.json")

total_time = time() - start_time

In [17]:
from akasha import prompts

def generate_recommend_titles(theme):
        formatter = prompts.JSON_formatter_list(names=["段落標題", "段落次標題"], types=["list", "list"], descriptions=["每段段落標題", "每段多個段落次標題"])
        JSON_prompt = prompts.JSON_formatter(formatter)
        generated_titles = QA.ask_self(
            system_prompt=JSON_prompt,
            prompt=f"我想要寫一份報告，請以{theme}主題，幫我制定四個或五個段落標題，其中每個段落標題都有其各自的次標題，請參考以下範例，並回答。",
            info="""
                範例:
                [給定主題]
                電池產業的發展趨勢
                [回應範例]
                全球電池市場概況

                市場規模與增長趨勢
                主要市場區域分析
                主要企業概況

                技術發展趨勢

                鋰離子電池技術進展
                固態電池技術
                其他新興電池技術
            """,
            model="openai:gpt-4"
        )
        print(generated_titles)


generate_recommend_titles("核能發電的未來:挑戰與機遇")

{
    "段落標題": ["核能發電的現狀", "核能發電的挑戰", "核能發電的機遇", "核能發電的未來趨勢", "結論"],
    "段落次標題": [
        ["全球核能發電概況", "臺灣核能發電現況"],
        ["核能發電的環境問題", "核能發電的安全問題", "核能發電的經濟問題"],
        ["核能發電的環保機遇", "核能發電的能源供應機遇"],
        ["新型核能發電技術", "核能發電的政策趨勢"],
        ["核能發電的未來展望", "建議與對策"]
    ]
}


In [ ]:
result["內容摘要"] = summary.summarize_articles(
    articles=previous_result,
    format_prompt=f"將內容以{theme}為主題進行摘要，將用字換句話說，意思不便，不需要結論，不需要回應要求。",
    summary_len=500
)

In [ ]:
print("----------Complete report----------")
# print("Total time: ", total_time, "s")
for key, value in result.items():
    print(key, ":\n\n\t", value, "\n")

In [ ]:
generate_report_tool = akasha.akashas.create_tool(
    "Generate Report",
    """
    Generate a report based on a given theme, titles, and links.

    Parameters:
    - theme (str): The main theme of the report
    - titles (Dict[str, str]): A dictionary of titles and subtitles
    - links (List[str]): A list of links to process

    Returns:
    - Dict[str, str]: A dictionary containing each title and its corresponding content
    """,
    func = generate_report
)

titles_input_tool = akasha.akashas.create_tool(
    "Titles Input",
    """
    Input the titles and subtitles of the report.

    Parameters:
    - theme (str): The main theme of the report

    Returns:
    - Dict[str, str]: A dictionary of titles and subtitles
    """,
    func = lambda theme: titles
)

## 修改文句工具

In [ ]:
part = input("想要修改哪一部分的內容? (輸入標題或\"結束\")\n")
while part != "結束":
    if part in result:
        previous_context = result[part]
        print(f"----------目前內容----------\n\n{previous_context}\n\n------------------------")
        new_prompt = input("想要如何修改:\n")
        new_response = QA.ask_self(
            prompt=f"""將此內容根據以下要求進行修改，若無法達成則不要修改任何內容直接輸出原始內容，不要亂撰寫內容:
            ----------------
            {new_prompt}
            ----------------

            範例:
            <原始內容>
            台灣的電池產業發展迅速，主要市場區域包括亞洲、美洲和歐洲。
            <要求>
            加入非洲市場區域分析。
            (無法達成)
            <修改後內容>
            "無法達成要求，因此不做任何修改"     <----- 這句話連同引號是必要的，以示未達成要求
            (空一行)                        <----- 這行是必要的，用來分隔原始內容和修改後內容
            台灣的電池產業發展迅速，主要市場區域包括亞洲、美洲和歐洲。
            """,
            info=previous_context,
            model="openai:gpt-4"
        )
        print(f"----------修改後內容----------\n\n{new_response}\n\n--------------------------")
        save_or_not = input("是否要儲存修改? (y/n)\n")
        if save_or_not == "y":
            result[part] = new_response
    else:
        print("找不到對應的標題")
    part = input("想要修改哪一部分的內容? (輸入標題或\"結束\")\n")

## 重新擷取網路文字工具

In [ ]:
command = input("請輸入修改要求")
new_request= QA.ask_self(
    prompt=f"""使用者輸入了以下修改要求:
    ----------------
    {command}
    ----------------
    從給定的list和使用者的修改要求中找出使用者要求修改的部分為何，以及要如何修改，前面不須加上"回覆:"。

    範例:
    <給定的list>
    ["全球電池市場概況", "技術發展趨勢", "應用領域與市場機會", "政策與供應鏈分析"]
    <要求>
    技術發展趨勢的部分我覺得不夠完整，請加入更多新興電池技術的內容。
    <回覆>
    修改部分: 技術發展趨勢
    修改內容: 加入更多新興電池技術的內容

    若要求的部分不在給定的list當中，請回覆"報告中無此部分，請確認後再提出修改要求"，前面不須加上"回覆:"。

    範例:
    <給定的list>
    ["全球電池市場概況", "技術發展趨勢", "應用領域與市場機會", "政策與供應鏈分析"]
    <要求>
    我覺得電池發展歷史的部分不夠完整，請幫我修改。
    <回覆>
    報告中無此部分，請確認後再提出修改要求。    <----- 只需回覆這句話，前面不須加上"回覆:"

    若修改要求中雖然有錯別字或文法錯誤，但仍能理解要求，也請回覆"修改部分: <修改部分> 修改內容: <修改內容>"，前面不須加上"回覆:"。

    範例:
    <給定的list>
    ["全球電池市場概況", "技術發展趨勢", "應用領域與市場機會", "政策與供應鏈分析"]
    <要求>
    我覺得全球電池試場概況的部分不夠完整，請加入更多市場規模與增長趨勢的內容。
    <回覆>
    修改部分: 全球電池市場概況
    修改內容: 加入更多市場規模與增長趨勢的內容

    若只有提出修改的要求，沒有指定要修改的部分，請回覆"不知道您想要修改哪一部分，請提供更多資訊"。避免自動回覆list中所有項目。前面不須加上"回覆:"。
    只要要求中沒有看到list中的element，就回覆"不知道您想要修改哪一部分，請提供更多資訊"。
    範例:
    <給定的list>
    ["全球電池市場概況", "技術發展趨勢", "應用領域與市場機會", "政策與供應鏈分析"]
    <要求>
    去除2025年以前的資料。  <----- 只有提出修改的要求，沒有指定要修改的部分
    <回覆>
    不知道您想要修改哪一部分，請提供更多資訊    <----- 需要修改的部分不明確，也沒有list中的element，因此只需回覆這句話，前面不須加上"回覆:"

    若無明確指定想要如何修改，回覆"無法理解您的修改要求，請提供更多資訊"，前面不須加上"回覆:"。

    <給定的list>
    ["全球電池市場概況", "技術發展趨勢", "應用領域與市場機會", "政策與供應鏈分析"]
    <要求>
    我覺得技術發展趨勢的部分有問題。
    <回覆>
    無法理解您的修改要求，請提供更多資訊    <----- 只需回覆這句話，前面不須加上"回覆:"
    """,
    info=[key for key in result.keys()],
    model="openai:gpt-4"
)
print(new_request)
try:
    part = new_request.split("修改部分: ")[1].split("\n修改內容: ")[0]
    mod_command = new_request.split("修改部分: ")[1].split("\n修改內容: ")[1]
    if part in result:
        previous_context = result[part]

        modification = QA.ask_self(
            prompt=f"""從以下修改要求和提供的內容判斷是否需要重新爬取資料。大部分情況需要加入新的資料進去時會，
            需要爬取資料，此時需要回覆"y"。否則回覆"n"。若修改要求和提供的內容完全沒有關聯性或者無法判斷時，請回覆"unknown"。
            ----------------
            {mod_command}
            ----------------
            y的範例:
            <修改要求>
            加入非洲市場區域分析。
            <提供的內容>
            台灣的電池產業發展迅速，主要市場區域包括亞洲、美洲和歐洲。
            <回覆>
            y

            n的範例:
            <修改要求>
            消去亞洲市場區域分析。
            <提供的內容>
            台灣的電池產業發展迅速，主要市場區域包括亞洲、美洲和歐洲。
            <回覆>
            n

            unknown的範例:
            <修改要求>
            加入非洲動物大遷徙的資訊
            <提供的內容>
            台灣的電池產業發展迅速，主要市場區域包括亞洲、美洲和歐洲。
            <回覆>
            unknown
            """,
            info=previous_context,
            model="openai:gpt-4"
        )

        while modification == "unknown":
            modification = input("無法判斷是否需要重新爬取資料，請問是否需要從原文重新爬取資料? (y/n)\n")
            if modification != "y" and modification != "n":
                modification = "unknown"
        if modification == "y":
            print("需重新爬取資料")
            new_response = generate_report(theme=theme, titles={part:[titles[part]]}, links=links, rerun_process=True)
        elif modification == "n":
            print("不需重新爬取資料，嘗試直接修改內容中......")
            new_response = QA.ask_self(
                prompt=f"""將此內容根據以下要求進行修改，若無法達成則不要修改任何內容直接輸出原始內容，不要亂撰寫內容:
                ----------------
                {mod_command}
                ----------------

                範例:
                <原始內容>
                台灣的電池產業發展迅速，主要市場區域包括亞洲、美洲和歐洲。
                <要求>
                加入非洲市場區域分析。
                (無法達成)
                <修改後內容>
                "無法達成要求，因此不做任何修改"     <----- 這句話連同引號是必要的，以示未達成要求
                (空一行)                        <----- 這行是必要的，用來分隔原始內容和修改後內容
                台灣的電池產業發展迅速，主要市場區域包括亞洲、美洲和歐洲。(輸出與原始內容相同的內容)
                """,
                info=previous_context,
                model="openai:gpt-4"
            )

        print(f"----------{part}的原本內容----------\n\n{previous_context}\n\n---------------------------------")
        print(f"----------{part}的修改後內容----------\n\n{new_response}\n\n-----------------------------------")
        save_or_not = input("是否要儲存修改? (y/n)\n")
        if save_or_not == "y":
            result[part] = new_response
except:
    print(new_request)